<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/MARCA-Color.jpg" title="Title text" width="50%" height="50%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales II-2018 </h1>

<H3 align='center'> Tarea 2 - Redes Convolucionales y sus aplicaciones </H3>
<hr style="height:2px;border:none"/>

Alunos: 

    Christian da Rocha Iardino - 201890251-9
    
    Rodrigo Hermozilla - 95714144-4

## 3. CNN sobre texto

##### Carga inicial de librerias
> En la siguiente sección y de manera previa se cargan todas las librerías necesarias para el ejercicio. Para efectos de mantener cierto orden con el código de las preguntas del ejercicio, se determino que todas las cargas se realizaran previamente, no obstante, se entiende que esto podría generar alguna ineficiencia en el consumo de recursos, pero que suponemos es marginal.

In [1]:
import pandas as pd
import numpy as np
import re, os, sys
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer

In [2]:
lmtz = WordNetLemmatizer()
stoplist = stopwords.words('english')
lmtzlist = [1]*len(stoplist)
jArray = [1]*len(stoplist)
count = 0

j = 0
while j <= len(stoplist)-1:
    lmtzlist[j] = lmtz.lemmatize(stoplist[j])
    j = j+1

    
j = 0
while j <= len(stoplist)-1:
    if lmtzlist[j] != stoplist[j]:
        jArray[count] = j
        count = count + 1
    
    j = j+1
    
count = count + len(stoplist)
newStopList = [1]*count

j = 0
while j<= count-1:
    if j <= len(stoplist)-1:
        newStopList[j] = stoplist[j]
    else:
        newStopList[j] = lmtzlist[jArray[count-len(stoplist)]]
    j = j+1

In [3]:
df = pd.read_csv("data/Train_rev1.csv")
df.drop(columns=['Id','LocationRaw','ContractType','SalaryRaw','SourceName'],axis=1,inplace=True)
df['Title'].fillna('untituled', inplace=True)
df['FullDescription'].fillna('undrecribe', inplace=True)
df['LocationNormalized'].fillna('unlocate', inplace=True)
df['ContractTime'].fillna('doubt', inplace=True)
df['Company'].fillna('unknown', inplace=True)
df['Category'].fillna('unspecified', inplace=True)
name_dict = {0 :'Title', 1 :'FullDescription', 2 :'LocationNormalized',  3 :'Category', 4:'ContractTime', 5:'Company' }
df.head()

,Title,FullDescription,LocationNormalized,ContractTime,Company,Category,SalaryNormalized
0,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,Dorking,permanent,Gregory Martin International,Engineering Jobs,25000
1,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,Glasgow,permanent,Gregory Martin International,Engineering Jobs,30000
2,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,Hampshire,permanent,Gregory Martin International,Engineering Jobs,30000
3,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,Surrey,permanent,Gregory Martin International,Engineering Jobs,27500
4,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...",Surrey,permanent,Gregory Martin International,Engineering Jobs,25000


<div class="alert alert-block alert-success">
    <b>Nota: </b>Para efectos de mejorar la modularización, el ejercicio construye una serie de funciones, permitiendo además su reutilización en distintos instantes.
</div>

In [4]:
def textos(x):
    texto = [None]*len(df[str(name_dict[x])])
    i = 0
    while i < len(df[str(name_dict[x])]):
        texto[i] = str(df[str(name_dict[x])][i])
        i += 1
    return texto

In [5]:
def pre_procesar(x):
    df_procesado = {}
    counter = 0
    for s in textos(x):
        s = s.lower()
        s = re.sub(r'[^\w]', ' ', s)
        s = re.sub(r'\b[a-z]\b', ' ', s)
        s = re.sub(r'\b[a-z][a-z]\b', ' ', s)
        s = re.sub(r'\b[0-9]\b', ' ',  s)
        s = re.sub(r'\b[0-9][0-9]\b', ' ', s)
        s = re.sub(r'\b[0-9][0-9][0-9]\b', ' ', s)
        s = re.sub(r'\b[0-9][0-9][0-9][0-9]\b', ' ', s)
        s = re.sub(r'\b[0-9][0-9][0-9][0-9][0-9]\b', ' ', s)
        s = re.sub(r'\b[0-9][0-9][0-9][0-9][0-9][0-9]\b', ' ', s)
        s = re.sub(r'[^\w.]', ' ', s)
        s = re.sub(r'\s+', ' ', s)
        s = s.split(',')
        df_procesado[counter] = s
        counter += 1
    return df_procesado 

In [6]:
def procesado(x):
    texto_procesado = str([ v for v in pre_procesar(x).values() ])
    texto_procesado = texto_procesado.replace('[[','[')
    texto_procesado = texto_procesado.replace(']]',']')
    texto_procesado = texto_procesado.replace('[','')
    texto_procesado = texto_procesado.replace("'",'')
    texto_procesado = texto_procesado.replace(",",'')
    return texto_procesado

In [7]:
def convert_to_list(x):
    string = procesado(x)
    convertido = string.split(']')
    return convertido

In [8]:
def conver_to_one_string():
    temp5 = convert_to_list(5)
    temp4 = convert_to_list(4)
    temp3 = convert_to_list(3)
    temp2 = convert_to_list(2)
    temp1 = convert_to_list(1)
    temp0 = convert_to_list(0)
    x = 0
    while x<len(temp0):
        temp0[x] = temp0[x]+' '+temp1[x]+' '+temp2[x]+' '+temp3[x]+' '+temp4[x]+' '+temp5[x]
        x+=1
    convert_list = temp0
    return convert_list  

In [9]:
def fazer_dicionario(x):
    dicionario = procesado(x)
    dicionario =  dicionario.replace(']','')
    dicionario = dicionario.split(' ')
    return dicionario

In [10]:
def junta_dicionario(x):
    k = x-1
    dic = fazer_dicionario(x)
    while k >= 0:
        dic = dic + fazer_dicionario(k)
        k = k-1
    return dic

> Descarga y asignación de variables estructuras globales

In [11]:
y_dataframe = df['SalaryNormalized'].values
x_dataframe = df[['Title','FullDescription','LocationNormalized','Category', 'ContractTime', 'Company' ]]

In [12]:
df_procesado = conver_to_one_string()

k = len(df_procesado)
x_train = df_procesado[:int(0.7*k)] 
x_val = df_procesado[int(0.70*k):int(0.85*k)]
x_test = df_procesado[int(k*0.85)::]

y_train = y_dataframe[:int(0.7*k)] 
y_val = y_dataframe[int(0.70*k):int(0.85*k)]
y_test = y_dataframe[int(k*0.85)::]

In [13]:
todas_palavras = junta_dicionario(5)

In [14]:
todas_palavras = set(todas_palavras)
todas_palavras = list(todas_palavras)

In [15]:
word_index = dict(enumerate(todas_palavras))
j=0
for frase in x_train:
    seq=frase.split()
    for term in seq:
        if term not in word_index.keys():
            word_index[term]=j
            j+=1

In [16]:
embeddings_index = {}
f = open(os.path.join('data/glove.6B.100d.txt'),encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Se encontraron %s terminos con sus vectores de embedding.' % len(embeddings_index))

Se encontraron 400000 terminos con sus vectores de embedding.


In [17]:
embedding_vector = 100
embedding_matrix = np.zeros((len(word_index.keys()), embedding_vector))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector  

In [18]:
from keras.preprocessing import sequence
x_new_train = [[word_index[word] for word in text.split()] for text in x_train]
x_new_val = [[word_index[word] for word in text.split() if word in word_index] for text in x_val]

max_input_lenght = 1000
Xtrain = sequence.pad_sequences(x_new_train,maxlen=max_input_lenght,padding='post',value=0)
Xval = sequence.pad_sequences(x_new_val,maxlen=max_input_lenght,padding='post',value=0)

Using TensorFlow backend.


In [19]:
from keras.layers import Embedding , Flatten , Input , Dense , BatchNormalization , Activation, Dropout
from keras.models import Model
from keras import optimizers
from keras.layers import Conv1D, MaxPooling1D
from sklearn.metrics import mean_absolute_error

In [ ]:
poo1 = 25
poo2 = 6
poo3 = 2
poo4 = 1
stride1 = 4
stride2 = 2
stride3 = 1
stride4 = 1
con = 128
con2 = 7

epo = 11

embedding_vector = 100

embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                        input_length=max_input_lenght,trainable=False)
sequence_input = Input(shape=(max_input_lenght,))
embedded_sequences = embedding_layer(sequence_input)

conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
conv4 = Conv1D(con, con2, activation='relu',padding='same')(pool4)
pool5 = MaxPooling1D(pool_size=poo4, stride=(stride4))(conv4)
flat = Flatten()(pool5)

preds = Dense(1, activation='linear')(flat)
model = Model(sequence_input, preds)
model.summary()

model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))

C:\Users\Christian\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=25, strides=4)`
C:\Users\Christian\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=6, strides=2)`
C:\Users\Christian\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2, strides=1)`
C:\Users\Christian\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=1, strides=1)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         36526900  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 128)         89728     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 244, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 244, 128)          114816    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 120, 128)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 120, 128)          114816    
__________

### Primera optimización

> Se realizó con 60 muestras del total debido al alto tiempo para el aprendizaje

####  Primera ronda de pruebas:

> En el caso de las pruebas con MAE menores de 12.000 se han documentado, realizado más de 350 pruebas, estas variables se utilizarán durante las pruebas, salvo que alguna prueba explique el cambio de algún valor.

Menor MAE on validation = 10.502,742

```python

y_dataframe = df['SalaryNormalized'].values

x_dataframe = df[['Title','FullDescription','LocationNormalized','ContractTime','Company',,'Category']]

```

X** Representa el mejor parámetro encontrado

##### Mejores parámetros revisados:

```python
poo1 = 15, 17, 19, 20, 21, 22, 23**, 24, 25

poo2 = 4, 6, 8, 10, 11**

poo3 = 1, 2, 3, 4**

stride1 = 8, 9, 10**,11, 12 , 15, 20, 25

stride2 = 1, 2, 3, 4**, 5, 6

stride3 = 1**

con = 64, 80, 110, 128**, 156, 189, 220, 228, 256

con2 = 2, 4, 5, 6, 7**, 8, 9

```

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Probado con valores diferentes en cada capa pero con valores iguales presenta mayor desempeño.
</div>

```python
epo = 5**, 6, 7, 10, 11, 12, 13, 15, 25 
```

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Pocos casos que necesitan más de 5 epoch para mejorar el rendimiento.
</div>

##### Mejores Activaciones:

```python

Relu* (testada com ativação final: Relu, Linear**, Elu, Selu, Softmax)

Elu (testada com ativação final: Relu, Linear, Elu, Selu, Softmax)

Selu (testada com ativação final: Relu, Linear, Elu, Selu, Softmax)

Softplux (testada com ativação final: Relu, Linear, Elu, Selu, Softmax)

Sigmoid (testada com ativação final: Linear, Relu)

Tanh (testada com ativação final: Linear, Relu)

Softmax (testada com ativação final: Linear)

Softsign (testada com ativação final: Linear, Relu)

Hard_sigmoid (testada com ativação final: Linear, Relu)
```

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Las pruebas apuntan que la variación de activación tiende a empeorar el rendimiento del código.
</div>

##### Mejores Optimizadores:

```python
RMSprop**

Adagrad

Adam

Adamax

Nadam
```

#### Segunda ronda de pruebas:

> Menor MAE on validation = 11.599,695

```pythion
y_dataframe = df['SalaryNormalized'].values

x_dataframe = df[['Title','FullDescription','LocationNormalized','Category', 'ContractTime', 'Company' ]]
```

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>La retirada de 'Contract Time' y 'Company' presentó una influencia significativa en el resultado final de MAE.
</div>

#### Tercera ronda de pruebas:

Menor MAE on validation Total = 10.596,260

> Cambiando las capas de las redes neuronales:

 $C \times P \times C \times P \times C \times P \times F$.

```pythion
conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
layerk = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
flat = Flatten()(layerk)
```
 Menor MAE on validation = 10.599,695

> Primeira cambio: $C \times C \times C \times P \times P \times P \times F$.

```pythion
conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(conv1)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv3)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(pool1)
layerk = MaxPooling1D(pool_size=poo3, stride=(stride3))(pool2)
flat = Flatten()(layerk)
```
 Menor MAE on validation = 12.361,527

> Segunda cambio: $C \times P \times C \times C \times P \times P \times F$.

```python
conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv3)
layerk = MaxPooling1D(pool_size=poo3, stride=(stride3))(pool2)
flat = Flatten()(layerk)
```
 Menor MAE on validation = 10.596,260

> Terceira cambio: $C \times P \times P \times C \times C \times P \times F$.

```python
conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(pool1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
layerk = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
flat = Flatten()(layerk)
```
 Menor MAE on validation = 10.942,665

> Cuarta cambio: $C \times P \times P \times P \times C \times C \times F$.

```pythion
conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(pool1)
layerk = MaxPooling1D(pool_size=poo3, stride=(stride3))(pool2)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(layerk)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
flat = Flatten()(conv3)
```
Menor MAE on validation = 10.982,401

> Quinta cambio: $P \times C \times P \times P \times C \times C \times F$.

```pythion
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(embedded_sequences)
conv1 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv1)
layerk = MaxPooling1D(pool_size=poo3, stride=(stride3))(pool2)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(layerk)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
flat = Flatten()(conv3)
```
 Menor MAE on validation = 10.658,778

> Sexta cambio: $P \times P \times C \times P \times C \times C \times F$.

```pythion
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(embedded_sequences)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(pool1)
conv1 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
layerk = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(layerk)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
flat = Flatten()(conv3)
```
 Menor MAE on validation = 10.658,778

*Cambio de epo para 25

> Setima cambio: $P \times P \times P \times C \times C \times C \times F$.

```pythion
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(embedded_sequences)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(pool1)
layerk = MaxPooling1D(pool_size=poo3, stride=(stride3))(pool2)
conv1 = Conv1D(con, con2, activation='relu',padding='same')(layerk)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(conv1)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
flat = Flatten()(conv3)

```
 Menor MAE on validation = 10.897,826

Cambio de epo para 25

> Oitava cambio: $P \times C \times P \times C \times P \times C \times F$.

```pythion
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(embedded_sequences)
conv1 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
layerk = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv2)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(layerk)
flat = Flatten()(conv3)

```
 Menor MAE on validation = 10.984,201
 
<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Los modelos que presentan repetición de capas presentan menor eficiencia.
</div>

#### Cuarta ronda de pruebas

Menor MAE on validation Total = 10.554,768

Adición de capas:

$C \times P \times C \times C \times P \times P \times F$.

```python
conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv3)
layerk = MaxPooling1D(pool_size=poo3, stride=(stride3))(pool2)
flat = Flatten()(layerk)
```

> Primeira cambio: $C \times P \times C \times C \times P \times P \times C \times C \times C \times P \times P \times P> \times F$.

```pythion
conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv3)
pool3 = MaxPooling1D(pool_size=poo3, stride=(stride3))(pool2)
conv4 = Conv1D(con, con2, activation='relu',padding='same')(pool3)
conv5 = Conv1D(con, con2, activation='relu',padding='same')(conv4)
conv6 = Conv1D(con, con2, activation='relu',padding='same')(conv5)
pool4 = MaxPooling1D(pool_size=1, stride=(1))(conv6)
pool5 = MaxPooling1D(pool_size=1, stride=(1))(pool4)
layerk = MaxPooling1D(pool_size=1, stride=(1))(pool5)
flat = Flatten()(layerk)
```
Menor MAE on validation = 13.188,347

> Segunda cambio: $C \times P \times C \times C \times P \times P \times C \times F$.

```pythion
conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv3)
pool3 = MaxPooling1D(pool_size=poo3, stride=(stride3))(pool2)
conv4 = Conv1D(con, con2, activation='relu',padding='same')(pool3)
flat = Flatten()(conv4)
```
Menor MAE on validation = 11.068,852

> Terceiro cambio: $C \times P \times C \times C \times P \times P \times P \times F$.

```pythion
conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv3)
pool3 = MaxPooling1D(pool_size=poo3, stride=(stride3))(pool2)
pool4 = MaxPooling1D(pool_size=1, stride=(1))(pool3)
flat = Flatten()(pool4)
```
Menor MAE on validation = 10.554,768

> Carta cambio: $C \times P \times C \times C \times P \times P \times P \times P \times F$.

```pythion
conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(conv2)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv3)
pool3 = MaxPooling1D(pool_size=poo3, stride=(stride3))(pool2)
pool4 = MaxPooling1D(pool_size=1, stride=(1))(pool3)
pool5 = MaxPooling1D(pool_size=1, stride=(1))(pool4)
flat = Flatten()(pool5)
```
Menor MAE on validation = 12.411,508

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>El umbral entre capas y Overfitting no queda claro, entonces esa prueba será ignorada si se realizan pruebas con todos los datos.
</div>

### Segunda optimización

Utilizando registro de los datos adquiridos por la Primera optimización y utilizando todo el conjunto


####  Primera ronda de pruebas:

```python
k = 0
while k<=6:
    
    poo1 = 23
    poo2 = 11
    poo3 = 4
    stride1 = 10
    stride2 = 4
    stride3 = 1
    con = 128
    con2 = 7
    epo = (k*2)+1

    embedding_vector = 100

    embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                            input_length=max_input_lenght,trainable=False)
    sequence_input = Input(shape=(max_input_lenght,))
    embedded_sequences = embedding_layer(sequence_input)

    conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
    pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
    conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
    pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
    conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
    pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
    flat = Flatten()(pool4)

    preds = Dense(1, activation='linear')(flat)
    model = Model(sequence_input, preds)
    model.summary()

    model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
    model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

    print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
    print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
    
    k += 1

```

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Mejor resultado es con epo = 11.
</div>

####  Segunda ronda de pruebas:

```python
k = 0
while k<=6:
    
    poo1 = (k*2)+17
    poo2 = 11
    poo3 = 4
    stride1 = 10
    stride2 = 4
    stride3 = 1
    con = 128
    con2 = 7
    epo = 12

    embedding_vector = 100

    embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                            input_length=max_input_lenght,trainable=False)
    sequence_input = Input(shape=(max_input_lenght,))
    embedded_sequences = embedding_layer(sequence_input)

    conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
    pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
    conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
    pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
    conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
    pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
    flat = Flatten()(pool4)

    preds = Dense(1, activation='linear')(flat)
    model = Model(sequence_input, preds)
    model.summary()

    model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
    model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

    print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
    print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
    
    k += 1

```

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Mejor resultado es con poo1 = 25.
</div>

#### Tercera ronda de pruebas:

```python
k = 0
while k<=6:
    
    poo1 = 26
    poo2 = (k*2)+8
    poo3 = 4
    stride1 = 10
    stride2 = 4
    stride3 = 1
    con = 128
    con2 = 7
    epo = 12

    embedding_vector = 100

    embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                            input_length=max_input_lenght,trainable=False)
    sequence_input = Input(shape=(max_input_lenght,))
    embedded_sequences = embedding_layer(sequence_input)

    conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
    pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
    conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
    pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
    conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
    pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
    flat = Flatten()(pool4)

    preds = Dense(1, activation='linear')(flat)
    model = Model(sequence_input, preds)
    model.summary()

    model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
    model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

    print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
    print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
    
    k += 1

```

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Mejor resultado es con poo2 = 6.
</div>

####  Cuarta ronda de pruebas:

```python
k = 0
while k<=3:
    
    poo1 = 26
    poo2 = 10
    poo3 = (k*2)+2
    stride1 = 10
    stride2 = 4
    stride3 = 1
    con = 128
    con2 = 7
    epo = 12

    embedding_vector = 100

    embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                            input_length=max_input_lenght,trainable=False)
    sequence_input = Input(shape=(max_input_lenght,))
    embedded_sequences = embedding_layer(sequence_input)

    conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
    pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
    conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
    pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
    conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
    pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
    flat = Flatten()(pool4)

    preds = Dense(1, activation='linear')(flat)
    model = Model(sequence_input, preds)
    model.summary()

    model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
    model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

    print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
    print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
    
    k += 1

```

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Mejor resultado es con poo3 = 2.
</div>

####  Quinta ronda de pruebas:

```python
k = 0
while k<=6:
    
    poo1 = 26
    poo2 = 10
    poo3 = 9
    stride1 = (k*2)+2
    stride2 = 4
    stride3 = 1
    con = 128
    con2 = 7
    epo = 12

    embedding_vector = 100

    embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                            input_length=max_input_lenght,trainable=False)
    sequence_input = Input(shape=(max_input_lenght,))
    embedded_sequences = embedding_layer(sequence_input)

    conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
    pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
    conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
    pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
    conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
    pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
    flat = Flatten()(pool4)

    preds = Dense(1, activation='linear')(flat)
    model = Model(sequence_input, preds)
    model.summary()

    model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
    model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

    print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
    print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
    
    k += 1

```

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Mejor resultado es con stride1 = 4.
</div>

####  Sexta ronda de pruebas:

```python
k = 0
while k<=6:
    
    poo1 = 26
    poo2 = 10
    poo3 = 9
    stride1 = 2
    stride2 = k
    stride3 = 1
    con = 128
    con2 = 7
    epo = 12
    
    embedding_vector = 100

    embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                            input_length=max_input_lenght,trainable=False)
    sequence_input = Input(shape=(max_input_lenght,))
    embedded_sequences = embedding_layer(sequence_input)

    conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
    pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
    conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
    pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
    conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
    pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
    flat = Flatten()(pool4)

    preds = Dense(1, activation='linear')(flat)
    model = Model(sequence_input, preds)
    model.summary()

    model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
    model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

    print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
    print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
    
    k += 1

```

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Mejor resultado es con stride2 = 2.
</div>

###  Terceira optimización:

Cambiando las capas de las redes neuronales

####  Primera ronda de pruebas:

> Probando con valores retirados de la Segunda optimización

```python
poo1 = 25
poo2 = 6
poo3 = 2
poo4 = 1
stride1 = 4
stride2 = 2
stride3 = 1
stride4 = 1
con = 128
con2 = 7
epo = 12

embedding_vector = 100

embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                        input_length=max_input_lenght,trainable=False)
sequence_input = Input(shape=(max_input_lenght,))
embedded_sequences = embedding_layer(sequence_input)

conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
conv4 = Conv1D(con, con2, activation='relu',padding='same')(pool4)
pool5 = MaxPooling1D(pool_size=poo4, stride=(stride4))(conv4)
flat = Flatten()(pool5)

preds = Dense(1, activation='linear')(flat)
model = Model(sequence_input, preds)
model.summary()

model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
```

MAE on train:  5.878,2438727853305

MAE on validation:  7.848,9840662716615

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Presenta una mejora significativa en relación a los experimentos realizados anteriormente.
</div>

####  Segunda ronda de pruebas:

```python
poo1 = 23
poo2 = 11
poo3 = 4
poo4 = 1
stride1 = 4
stride2 = 3
stride3 = 2
stride4 = 1
con = 128
con2 = 7
epo = 9

embedding_vector = 100

embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                        input_length=max_input_lenght,trainable=False)
sequence_input = Input(shape=(max_input_lenght,))
embedded_sequences = embedding_layer(sequence_input)

conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
conv4 = Conv1D(con, con2, activation='relu',padding='same')(pool4)
pool5 = MaxPooling1D(pool_size=poo4, stride=(stride4))(conv4)
flat = Flatten()(pool5)

preds = Dense(1, activation='linear')(flat)
model = Model(sequence_input, preds)
model.summary()

model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
```
MAE on train:  6.126,312721245228

MAE on validation:  7.932,425457653122

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Probablemente falta de entrenamiento o overfitting.
</div>

####  Terceiro ronda de pruebas:

```python
poo1 = 23
poo2 = 11
poo3 = 4
poo4 = 2
stride1 = 4
stride2 = 3
stride3 = 2
stride4 = 1
con = 128
con2 = 7
epo = 8

embedding_vector = 100

embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                        input_length=max_input_lenght,trainable=False)
sequence_input = Input(shape=(max_input_lenght,))
embedded_sequences = embedding_layer(sequence_input)

conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
conv4 = Conv1D(con, con2, activation='relu',padding='same')(pool4)
pool5 = MaxPooling1D(pool_size=poo4, stride=(stride4))(conv4)
flat = Flatten()(pool5)

preds = Dense(1, activation='linear')(flat)
model = Model(sequence_input, preds)
model.summary()

model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
```
MAE on train:  7.285,367916636041

MAE on validation:  8.758,202896293133

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>La segunda ronda de ensayo no sufrió de overfitting.
</div>

####  Cuarta ronda de pruebas:

```python
poo1 = 25
poo2 = 6
poo3 = 2
poo4 = 3
stride1 = 4
stride2 = 2
stride3 = 1
stride4 = 1
con = 128
con2 = 7
epo = 9

embedding_vector = 100

embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                            input_length=max_input_lenght,trainable=False)
sequence_input = Input(shape=(max_input_lenght,))
embedded_sequences = embedding_layer(sequence_input)

conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
conv4 = Conv1D(con, con2, activation='relu',padding='same')(pool4)
pool5 = MaxPooling1D(pool_size=poo4, stride=(stride4))(conv4)
flat = Flatten()(pool5)

preds = Dense(1, activation='linear')(flat)
model = Model(sequence_input, preds)
model.summary()

model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
    
```
MAE on train:  6.589,785485364785

MAE on validation:  7.551,125863574896

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>La primeira ronda de ensayo no sufrió de falta de entreinameinto.
</div>

####  Quinta ronda de pruebas:

```python
max_input_lenght = 500

poo1 = 25
poo2 = 6
poo3 = 2
poo4 = 1
stride1 = 4
stride2 = 2
stride3 = 1
stride4 = 1
con = 128
con2 = 7

epo = 12

embedding_vector = 100

embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                        input_length=max_input_lenght,trainable=False)
sequence_input = Input(shape=(max_input_lenght,))
embedded_sequences = embedding_layer(sequence_input)

conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
conv4 = Conv1D(con, con2, activation='relu',padding='same')(pool4)
pool5 = MaxPooling1D(pool_size=poo4, stride=(stride4))(conv4)
flat = Flatten()(pool5)

preds = Dense(1, activation='linear')(flat)
model = Model(sequence_input, preds)
model.summary()

model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
    
```
MAE on train:  5.117,542429402952

MAE on validation:  7.254,130028545468

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Lo cambio em max_input_lenght apresenta mejoras en lo MAE.
</div>

#### Sexta ronda de pruebas:

```python
max_input_lenght = 1000

poo1 = 25
poo2 = 6
poo3 = 2
poo4 = 1
stride1 = 4
stride2 = 2
stride3 = 1
stride4 = 1
con = 128
con2 = 7

epo = 11

embedding_vector = 100

embedding_layer = Embedding(input_dim=len(word_index.keys()),output_dim=embedding_vector,weights=[embedding_matrix],
                        input_length=max_input_lenght,trainable=False)
sequence_input = Input(shape=(max_input_lenght,))
embedded_sequences = embedding_layer(sequence_input)

conv1 = Conv1D(con, con2, activation='relu',padding='same')(embedded_sequences)
pool1 = MaxPooling1D(pool_size=poo1, stride = (stride1))(conv1)
conv2 = Conv1D(con, con2, activation='relu',padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=poo2, stride=(stride2))(conv2)
conv3 = Conv1D(con, con2, activation='relu',padding='same')(pool2)
pool4 = MaxPooling1D(pool_size=poo3, stride=(stride3))(conv3)
conv4 = Conv1D(con, con2, activation='relu',padding='same')(pool4)
pool5 = MaxPooling1D(pool_size=poo4, stride=(stride4))(conv4)
flat = Flatten()(pool5)

preds = Dense(1, activation='linear')(flat)
model = Model(sequence_input, preds)
model.summary()

model.compile(loss='mse',optimizer='RMSprop',metrics=['acc'])
model.fit(Xtrain, y_train, validation_data=(Xval, y_val),epochs=epo, batch_size=256)

print("MAE on train: ",mean_absolute_error(y_train, model.predict(Xtrain)))
print("MAE on validation: ",mean_absolute_error(y_val, model.predict(Xval)))
    
```

Hecho 10 preuebas:

Menor MAE on train:  5.297,.233496228831

Menor MAE on validation:  7.079,602934342231

Mayor MAE on train:  5.891,291799577192

Mayor MAE on validation:  7.429,655660591401

<div class="alert alert-block alert-success">
    <b>Conclusión: </b>Un cambio en el max_input_length y presentó MAE razonables, entonces para confirmación de la teoría realicé pero las pruebas no sufren una variación superior del 2.5% , pero creo que para tener un MAE por debajo de 7000 es necesario redes más complejas y profundas y posiblemente también más datos. También creo que los parámetros se han optimizado a un punto extremo satisfactorio.
</div>